In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [ ]:
# Set random seeds for reproducibility
torch.manual_seed(42)

In [ ]:
df = pd.read_csv('dataset/fmnist_small.csv')
df.head()

In [ ]:
# Create a 4x4 grid of images
fig, axes = plt.subplots(4, 4, figsize=(10, 10))
fig.suptitle("First 16 Images", fontsize=16)

# Plot the first 16 images from the dataset
for i, ax in enumerate(axes.flat):
    img = df.iloc[i, 1:].values.reshape(28, 28)  # Reshape to 28x28
    ax.imshow(img)  # Display in grayscale
    ax.axis('off')  # Remove axis for a cleaner look
    ax.set_title(f"Label: {df.iloc[i, 0]}")  # Show the label

plt.tight_layout(rect=[0, 0, 1, 0.96])  # Adjust layout to fit the title
plt.show()


In [ ]:
# train test split
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# scaling the feautures
X_train = X_train/255.0
X_test = X_test/255.0

In [ ]:
X_train

In [ ]:
# create CustomDataset Class
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)
    def __len__(self):
        return len(self.features)
    def __getitem__(self, index):
        return self.features[index], self.labels[index]

In [ ]:
# create train_dataset object
train_dataset = CustomDataset(X_train, y_train)

In [ ]:
train_dataset[0]

In [ ]:
# create test_dataset object
test_dataset = CustomDataset(X_test, y_test)

In [ ]:
# create train and test loader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False) # No need

In [ ]:
# define NN class

class MyNN(nn.Module):

    def __init__(self, num_features):

        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(num_features, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
            # output softmax automatically handled in nn.CrossEntropyLoss
        )

    def forward(self, x):
        return self.model(x)

In [ ]:
# set learning rate and epochs
epochs = 100
learning_rate = 0.1

In [ ]:
# instantiate the model
model = MyNN(X_train.shape[1])

# loss function
criterion = nn.CrossEntropyLoss()

# optimizer
optimizer = optim.SGD(model.parameters(), lr= learning_rate)

In [ ]:
len(train_loader) # 150 batches * 32 in each batch = 4800 total sample images

In [ ]:
# training loop

for epoch in range(epochs): # total 100 epoch * 150 batches = 15,000 times loop will calculate loss ?
    total_epoch_loss = 0
    for batch_features, batch_labels in train_loader:
        # forward pass
        outputs = model(batch_features)
        # calculate loss
        loss = criterion(outputs, batch_labels)
        # back pass
        optimizer.zero_grad()
        loss.backward()
        # update grads
        optimizer.step()
        total_epoch_loss = total_epoch_loss + loss.item()
    
    avg_loss = total_epoch_loss/len(train_loader)
    print(f'Epoch: {epoch+1 : >3} , Loss: {avg_loss : .5f}')

In [ ]:
# set model to eval mode - like self.train(False) | must use for BatchNorm Dropout -> turned off from here
model.eval()

In [ ]:
# evaluation code
total = 0
correct = 0

with torch.no_grad():

  for batch_features, batch_labels in test_loader:

    outputs = model(batch_features)

    _, predicted = torch.max(outputs, 1) # 1 result in entire row

    total = total + batch_labels.shape[0]

    correct = correct + (predicted == batch_labels).sum().item()

print(correct/total)


More accuracy? And faster training on entire dataset? => Go with GPU